# Basics of Mobile Robotics - Final Project

**Students: Cécile Chavane, Yucef Grebici, Camille Guillaume, Théo Hermann**

<h1>Table of Contents<span class="tocSkip"></span></h1>

<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id=">Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item">
        <li><span><a href="#Project-description" data-toc-modified-id="Project-description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Project description</a></span></li>
        <li><span><a href="#Environment-Setup" data-toc-modified-id="Environment-Setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Environment Setup</a></span></li>
        <li><span><a href="#Libraries-Import" data-toc-modified-id="Libraires-Import-1.2"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries Import</a></span></li>
        <li><span><a href="#Thymio COnnection" data-toc-modified-id="Thymio-Connection-1.3"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Thymio Connection</a></span></li>
        </ul></li>
    <li><span><a href="#Description-of-the-modules" data-toc-modified-id="Description-of-the-modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Description of the Modules</a></span><ul class="toc-item">
        <li><span><a href="#Vision" data-toc-modified-id="Vision-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Vision</a></span></li>
        <li><span><a href="#Global-Navigation" data-toc-modified-id="Global-Navigation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Global Navigation</a></span></li>
        <li><span><a href="#Local-Navigation" data-toc-modified-id="Local-Navigation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Local Navigation</a></span></li>
        <li><span><a href="#Filtering" data-toc-modified-id="Filtering-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Filtering</a></span></li>
        <li><span><a href="#Motion-Control" data-toc-modified-id="Motion-Control-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Motion Control</a></span></li>
        </ul></li>
    <li><span><a href="#Code-of-the-overall-project" data-toc-modified-id="Code-of-the-overall-project-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Code of the overall project</a></span></li>
    <li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusion</a></span>
    </li></ul>
    
</div>

# Introduction

## Project description 


**Guidelines:**


**Implementation:**

## Environment  Setup

Description of the environment , obstacles,corners, start/goal, wireless thymio, camera position

<figure>
  <img src="pics/environment.jpeg" width="500" alt="Environment" />
  <figcaption> <center> <u>Figure 1:</u> Picture of the environment setup with lightning <center> </figcaption>
</figure>



<!-- Commentaire : <img src="pics/environment.jpeg" width="500"/> -->

## Libraries import

In [ ]:
#checkez ceux qui sont réellement nécéssaires
import numpy as np
import pyvisgraph as vg
import math
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point, LineString
import time

## Thymio Connection

In [ ]:
#communication asynchrone avec thymio
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

# Description of the modules

We separated our code in different python file containing the functions for each module. We will import all the python files below

In [ ]:
import Vision as vis
import Global as glob
import local as loc
import Filtering as fil
import control as ctrl

## Vision

## Global Navigation

## Local Navigation

## Filtering

## Motion Control

# Code of the overall Project

# Conclusion